# Set Up

In [ ]:
library(tidyverse)
library(fst)
library(bigrquery)
library(stringr)
library(lubridate)

# Acidosis Cohort

## Set Up

In [ ]:
# This snippet assumes that you run setup first

# This code copies a file from your Google Bucket into a dataframe

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'acidosis_emergent_conditions_AG_10172023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

# Load the file into a dataframe
acidosis.emergent  <- read_csv(name_of_file_in_bucket)
head(acidosis.emergent)

In [ ]:
# Columns that are needed for propensity score:
# 1. case vs control 1 and 0
# 2. basic survey date
# 3. earliest time of diagnosis of ANYTHING
# 4. total amount of diagnosis BEFORE basic survey date
# 5. any other columns that are needed in order to calculate 2-4

## Treatment Column

In [ ]:
# Create new df with just PIDs
acidosis_pid <- select(acidosis.emergent, c('PERSON_ID'))
dim(acidosis_pid)
head(acidosis_pid)

In [ ]:
acidosis <- acidosis_pid %>% 
       rename(person_id = PERSON_ID)
head(acidosis)
dim(acidosis)

In [ ]:
# Create new column for treatment (1) or control (0)
acidosis <- acidosis %>%
  mutate(Treatment = 1)
head(acidosis)
dim(acidosis)

## Basic Survey Column

In [ ]:
# Find the basic survey date for the PIDs in this dataframe

In [ ]:
# This snippet assumes that you run setup first

 

# This code copies a file from your Google Bucket into a dataframe

 

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'AUD_Survey_Basics_Lifestyle.fst'

 

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

 

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

 

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

 

# Load the file into a dataframe
aud_basics  <- read_fst(name_of_file_in_bucket)
head(aud_basics)

In [ ]:
unique(aud_basics$survey)
table(aud_basics['survey'])

In [ ]:
# create data frame with just the basics data
the.basics <- aud_basics[aud_basics$survey == 'The Basics',]
head(the.basics)
dim(the.basics)

In [ ]:
# See how many unique PID's are in this dataframe
length(unique(the.basics$person_id))

In [ ]:
# Remove duplicate PIDs
the.basics <- the.basics %>% distinct(person_id, .keep_all = TRUE)
head(the.basics)
dim(the.basics)

In [ ]:
# Keep the only column that is needed (survey_datetime)
basics <- select(the.basics, c('person_id', 'survey_datetime'))
dim(basics)
head(basics)

# Control Cohort

## Set Up

In [ ]:
# This snippet assumes that you run setup first

# This code copies a file from your Google Bucket into a dataframe

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'acidosis_control_cohort_pid_AG_11022023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

# Load the file into a dataframe
control  <- read_csv(name_of_file_in_bucket)
head(control)

## Treatment

In [ ]:
# Create new column for treatment (1) or control (0)
control <- control %>%
  mutate(Treatment = 0)
head(control)
dim(control)

# Acid + Control Merged

In [ ]:
head(acidosis)
dim(acidosis)

In [ ]:
head(control)
dim(control)

In [ ]:
# Combine the two dataframes 
combined <- rbind(acidosis, control)
combined
dim(combined)

## Basic Survey Column

In [ ]:
# Merge basics df and combined df, keeping only the PIDs from combined df
merged_basics <- merge(combined, basics, by="person_id", all.x = TRUE)
head(merged_basics)
dim(merged_basics)

In [ ]:
# Change survey_datetime to date
merged_basics$survey_date <- as.Date(merged_basics$survey_datetime)
head(merged_basics)

## Conditions Df

In [ ]:
# This snippet assumes that you run setup first

# This code copies a file from your Google Bucket into a dataframe

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'Conditions_Before_Enroll_CX_11032023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

# Load the file into a dataframe
df_condition  <- read_csv(name_of_file_in_bucket)
head(df_condition)

In [ ]:
dim(df_condition)

In [ ]:
# Merge the df_condition and the merged_basics dataframes

propensity <- merge(merged_basics, df_condition, by="person_id", all.x = TRUE)
head(propensity)
dim(propensity)

In [ ]:
# Find missing values and remove them
sum(is.na(propensity$num_diagnosis))

In [ ]:
is_na_num_diagnosis <- propensity[is.na(propensity$num_diagnosis),]
head(is_na_num_diagnosis)

In [ ]:
# See if these patients are part of the treatment or control group

table(is_na_num_diagnosis$Treatment)

In [ ]:
propensity_complete <- na.omit(propensity)
dim(propensity_complete)
head(propensity_complete)

In [ ]:
# Double check that the NA values are gone:
sum(is.na(propensity_complete$num_diagnosis))
sum(is.na(propensity_complete$min_condition_date))

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a fst file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- propensity_complete

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'propensity_matching_df_11032023AG.fst'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_fst(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.fst"), intern=T)

# Propensity Matching

In [ ]:
install.packages("MatchIt")

In [ ]:
library(MatchIt)
library(lmtest)
library(sandwich)

In [ ]:
# Upload the propensity dataframe

# This snippet assumes that you run setup first

# This code copies a file from your Google Bucket into a dataframe

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'propensity_matching_df_11032023AG.fst'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

# Load the file into a dataframe
propensity_complete  <- read_fst(name_of_file_in_bucket)
head(propensity_complete)

In [ ]:
dim(propensity_complete)

## Obtain matchit() basics

In [ ]:
# Run propensity scoring
match_obj <- matchit(Treatment ~ survey_date + min_condition_date + num_diagnosis,
                    data = propensity_complete, method = "nearest", distance = "glm",
                    ratio = 4,
                    replace = FALSE)
summary(match_obj)

In [ ]:
plot(match_obj, type = "jitter")
plot(match_obj, type = "hist")

In [ ]:
str(match_obj)

In [ ]:
match.matrix <- match_obj$match.matrix
match.matrix

# try and figure out what the columns are named

In [ ]:
summary(as.integer(row.names(match.matrix)))

In [ ]:
summary(as.vector(as.integer((match.matrix))))

In [ ]:
case_with_4controls_df <- data.frame(Cases = propensity_complete$person_id[as.integer(row.names(match.matrix))], 
                                     Control_1 =propensity_complete$person_id[as.integer(match.matrix[,1])],
                                     Control_2 =propensity_complete$person_id[as.integer(match.matrix[,2])],
                                     Control_3 =propensity_complete$person_id[as.integer(match.matrix[,3])],
                                     Control_4 =propensity_complete$person_id[as.integer(match.matrix[,4])])
                                     

In [ ]:
head(case_with_4controls_df)
dim(case_with_4controls_df)

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a fst file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- case_with_4controls_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'case_with_4controls_df_11032023_AG.fst'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_fst(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.fst"), intern=T)

## Patient Control List

In [ ]:
patient_controls <- as.character(as.vector(match.matrix))

In [ ]:
patient_controls <- propensity_complete$person_id[as.numeric(as.vector(match.matrix))]
head(patient_controls)

In [ ]:
length(unique(patient_controls))
# We now that the length of this list is the amount of control patients -- on the right
## track!

In [ ]:
summary(propensity_complete$person_id)

In [ ]:
summary(as.numeric(patient_controls))

## Patient Case List

In [ ]:
rownames(propensity_complete) <- propensity_complete$person_id
head(propensity_complete)

In [ ]:
patient_cases <- as.character(propensity_complete$person_id[propensity_complete$Treatment == 1])

In [ ]:
head(patient_cases)

In [ ]:
length(unique(patient_cases))
# We now that the length of this list is the amount of case patients -- on the right
## track!

In [ ]:
head(propensity_complete[patient_cases,])

## Unmatched Control List

In [ ]:
# GOAL: do set diff for the unmatched (patient id - person id cases - person id control)

In [ ]:
propensity_pid <- select(propensity_complete, c('person_id'))
dim(propensity_pid)

In [ ]:
head(propensity_pid)

In [ ]:
df_patient_controls <- as.data.frame(patient_controls)
head(df_patient_controls)

In [ ]:
colnames(df_patient_controls)[1]<-"person_id"
head(df_patient_controls)

In [ ]:
# Set diff: take out the controls
unmatched_controls <- setdiff(propensity_pid, df_patient_controls)
head(unmatched_controls)
dim(unmatched_controls)

In [ ]:
# 237786 - 227122 = 10664
## 10664 is the number of matched controls - function worked!

In [ ]:
# Set diff: take out the cases
df_patient_cases <- as.data.frame(patient_cases)
head(df_patient_cases)

In [ ]:
colnames(df_patient_cases)[1]<-"person_id"
head(df_patient_cases)

In [ ]:
# Need to change chr to dbl
df_patient_cases$person_id <- as.numeric(as.character(df_patient_cases$person_id))
head(df_patient_cases)

In [ ]:
unmatched_controls <- setdiff(unmatched_controls, df_patient_cases)
head(unmatched_controls)
dim(unmatched_controls)

In [ ]:
## 227122 - 224456 = 2666
## This is the number of cases in our cohort, so we are successful!

In [ ]:
# Create a new dataframe with propensity complete that only contains PIDs from 
## unmatched controls
unmatched_controls_df <- merge(unmatched_controls, propensity_complete, 
                               by="person_id", all.x = TRUE)
head(unmatched_controls_df)
dim(unmatched_controls_df)

## Dfs: Control/Case/Unmatched

In [ ]:
head(propensity_complete)
dim(propensity_complete)

In [ ]:
# #'s = controls: 10664, case: 2666, unmatched: 224456

In [ ]:
# cases df

In [ ]:
head(df_patient_cases)
dim(df_patient_cases)

In [ ]:
case_propensity_df <- merge(df_patient_cases, propensity_complete, 
                               by="person_id", all.x = TRUE)
head(case_propensity_df)
dim(case_propensity_df)

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a fst file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- case_propensity_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'cases_propensity_df_11032023_AG.fst'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_fst(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.fst"), intern=T)

In [ ]:
# controls df 

In [ ]:
head(df_patient_controls)
dim(df_patient_controls)

In [ ]:
control_propensity_df <- merge(df_patient_controls, propensity_complete, 
                               by="person_id", all.x = TRUE)
head(control_propensity_df)
dim(control_propensity_df)

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a fst file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- control_propensity_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'control_propensity_df_11032023_AG.fst'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_fst(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.fst"), intern=T)

In [ ]:
# unmatched controls
head(unmatched_controls_df)
dim(unmatched_controls_df)

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a fst file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- unmatched_controls_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'unmatched_propensity_df_11032023_AG.fst'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_fst(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.fst"), intern=T)

# Descriptive Statistics

## Number of Diagnoses

In [ ]:
# Plot histogram: Patient CONTROLS Number of Diagnoses
hist(propensity_complete[as.character(patient_controls),'num_diagnosis'],
    main = "Patient Controls: # of Diagnoses",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

In [ ]:
# Plot histogram: Patient CASES Number of Diagnoses
hist(propensity_complete[as.character(patient_cases),'num_diagnosis'],
    main = "Patient Cases: # of Diagnoses",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

In [ ]:
# Plot histogram: UNMATCHED CONTROLS Number of Diagnoses
hist(unmatched_controls_df$num_diagnosis,
    main = "Unmatched Controls: # of Diagnoses",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

In [ ]:
# Find the summary for the # diagnoses for unmatched_controls_df
summary(unmatched_controls_df$num_diagnosis)

In [ ]:
# Plot histogram: UNMATCHED CONTROLS Number of Diagnoses - BUT REMOVE OUTLIER
hist(unmatched_controls_df$num_diagnosis
     [unmatched_controls_df$num_diagnosis <= 1200],
    main = "Unmatched Controls: # of Diagnoses",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

## Basic Survey Date

In [ ]:
# Plot histogram: Patient CONTROLS Basic Survey Date
hist(propensity_complete[as.character(patient_controls),'survey_date'],
    main = "Patient Controls: Basic Survey Date",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

In [ ]:
# Plot histogram: Patient CASES Basic Survey Date
hist(propensity_complete[as.character(patient_cases),'survey_date'],
    main = "Patient Cases: Basic Survey Date",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

In [ ]:
# Plot histogram: UNMATCHED CONTROLS Basic Survey Date
hist(unmatched_controls_df$survey_date,
    main = "Unmatched Controls: Basic Survey Date",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

## First Dx Date

In [ ]:
# Plot histogram: Patient CONTROLS Min Dx Date
hist(propensity_complete[as.character(patient_controls),'min_condition_date'],
    main = "Patient Controls: First Dx Date",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

In [ ]:
# This snippet assumes that you run setup first

 

# This code copies a file from your Google Bucket into a dataframe

 

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'control_propensity_df_11032023_AG.fst'

 

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

 

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

 

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

 

# Load the file into a dataframe
control  <- read_fst(name_of_file_in_bucket)
head(control)

In [ ]:
min(control[,6])

In [ ]:
# Plot histogram: Patient CASES Min Dx Date
hist(propensity_complete[as.character(patient_cases),'min_condition_date'],
    main = "Patient Cases: First Dx Date",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

In [ ]:
# This snippet assumes that you run setup first

 

# This code copies a file from your Google Bucket into a dataframe

 

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'cases_propensity_df_11032023_AG.fst'

 

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

 

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

 

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

 

# Load the file into a dataframe
cases  <- read_fst(name_of_file_in_bucket)
head(cases)

In [ ]:
min(cases[,6])

In [ ]:
# Plot histogram: UNMATCHED CONTROLS Min Dx Date
hist(unmatched_controls_df$min_condition_date,
    main = "Unmatched Controls: First Dx Date",
    col = "light blue",
    xlab = "Count",
    breaks = 100)

# Demographics

In [ ]:
# This snippet assumes that you run setup first

 

# This code copies a file from your Google Bucket into a dataframe

 

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'demographic_all.csv'

 

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

 

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

 

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

 

# Load the file into a dataframe
my_dataframe  <- read_csv(name_of_file_in_bucket)
head(my_dataframe)

In [ ]:
dim(my_dataframe)

## Controls + Demo


In [ ]:
# MERGING PATIENT CONTROLS AND DEMO DATA

In [ ]:
head(df_patient_controls)

In [ ]:
controls_demo_df <- merge(df_patient_controls, my_dataframe, 
                               by="person_id", all.x = TRUE)
head(controls_demo_df)
dim(controls_demo_df)

In [ ]:
# Clean demographic data
clean_controls_demo_df <- plyr::rename(controls_demo_df, c(person_id = 'Count',
#                                        year_of_birth = 'Age',
                                       race_source_value = 'Race',
                                       sex_at_birth_source_value = 'Sex_at_Birth',
                                       ethnicity_source_value = 'Hispanic',
                                       gender_source_value = 'Gender'))

In [ ]:
head(clean_controls_demo_df)

In [ ]:
for (row in 1:nrow(clean_controls_demo_df))
    {
    for (col in 1:ncol(clean_controls_demo_df))
        {
        if(grepl('PMI: Skip',clean_controls_demo_df[row,col]))
            {clean_controls_demo_df[row,col] <- "Skip"}
        if(clean_controls_demo_df[row,col] %in% c("Not man only, not woman only, prefer not to answer, or skipped",
                                          "No matching concept",
                                          "None of these",
                                          "I prefer not to answer"))
            {clean_controls_demo_df[row,col] <- 'Unspecified'}
    }
}

In [ ]:
today <- Sys.Date()
today <- format(today, format="%Y")
today

In [ ]:
clean_controls_demo_df$Age <- as.numeric(today) - as.numeric(clean_controls_demo_df$year_of_birth)

In [ ]:
breaks <- c(18,25,35,45,55,65,75,85,1000)
tags <- c('18-25','26-35','36-45','46-55','56-65','66-75','76-85','86+')
clean_controls_demo_df$Age_Group <- cut(clean_controls_demo_df$Age, breaks=breaks, right=FALSE, labels=tags)

In [ ]:
head(clean_controls_demo_df)

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a fst file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- clean_controls_demo_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'controls_demo_df_11032023_AG.fst'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_fst(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.fst"), intern=T)

## Cases + Demo

In [ ]:
# MERGING PATIENT CASES AND DEMO DATA

In [ ]:
head(df_patient_cases)

In [ ]:
# This snippet assumes that you run setup first

 

# This code copies a file from your Google Bucket into a dataframe

 

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'demographic_all.csv'

 

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

 

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

 

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

 

# Load the file into a dataframe
my_dataframe1  <- read_csv(name_of_file_in_bucket)
head(my_dataframe1)

In [ ]:
cases_demo_df <- merge(df_patient_cases, my_dataframe1, 
                               by="person_id", all.x = TRUE)
head(cases_demo_df)
dim(cases_demo_df)

In [ ]:
# Clean cases data

# Clean demographic data
clean_cases_demo_df <- plyr::rename(cases_demo_df, c(person_id = 'Count',
#                                        year_of_birth = 'Age',
                                       race_source_value = 'Race',
                                       sex_at_birth_source_value = 'Sex_at_Birth',
                                       ethnicity_source_value = 'Hispanic',
                                       gender_source_value = 'Gender'))
for (row in 1:nrow(clean_cases_demo_df))
    {
    for (col in 1:ncol(clean_cases_demo_df))
        {
        if(grepl('PMI: Skip',clean_cases_demo_df[row,col]))
            {clean_cases_demo_df[row,col] <- "Skip"}
        if(clean_cases_demo_df[row,col] %in% c("Not man only, not woman only, prefer not to answer, or skipped",
                                          "No matching concept",
                                          "None of these",
                                          "I prefer not to answer"))
            {clean_cases_demo_df[row,col] <- 'Unspecified'}
    }
}

clean_cases_demo_df$Age <- as.numeric(today) - as.numeric(clean_cases_demo_df$year_of_birth)

breaks <- c(18,25,35,45,55,65,75,85,1000)
tags <- c('18-25','26-35','36-45','46-55','56-65','66-75','76-85','86+')
clean_cases_demo_df$Age_Group <- cut(clean_cases_demo_df$Age, breaks=breaks, right=FALSE, labels=tags)

In [ ]:
head(clean_cases_demo_df)

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a fst file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- clean_cases_demo_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'cases_demo_df_11032023_AG.fst'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_fst(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.fst"), intern=T)

## Unmatched + Demo

In [ ]:
# MERGING UNMATCHED CONTROLS AND DEMO DATA

In [ ]:
head(unmatched_controls)

In [ ]:
unmatched_controls_demo_df <- merge(unmatched_controls, my_dataframe1, 
                               by="person_id", all.x = TRUE)
head(unmatched_controls_demo_df)
dim(unmatched_controls_demo_df)

In [ ]:
# Clean unmatched control data

# Clean demographic data
clean_unmatched_demo_df <- plyr::rename(unmatched_controls_demo_df, 
                                                 c(person_id = 'Count',
#                                        year_of_birth = 'Age',
                                       race_source_value = 'Race',
                                       sex_at_birth_source_value = 'Sex_at_Birth',
                                       ethnicity_source_value = 'Hispanic',
                                       gender_source_value = 'Gender'))
for (row in 1:nrow(clean_unmatched_demo_df))
    {
    for (col in 1:ncol(clean_unmatched_demo_df))
        {
        if(grepl('PMI: Skip',clean_unmatched_demo_df[row,col]))
            {clean_unmatched_demo_df[row,col] <- "Skip"}
        if(clean_unmatched_demo_df[row,col] %in% c("Not man only, not woman only, prefer not to answer, or skipped",
                                          "No matching concept",
                                          "None of these",
                                          "I prefer not to answer"))
            {clean_unmatched_demo_df[row,col] <- 'Unspecified'}
    }
}

clean_unmatched_demo_df$Age <- as.numeric(today) - as.numeric(clean_unmatched_demo_df$year_of_birth)

breaks <- c(18,25,35,45,55,65,75,85,1000)
tags <- c('18-25','26-35','36-45','46-55','56-65','66-75','76-85','86+')
clean_unmatched_demo_df$Age_Group <- cut(clean_unmatched_demo_df$Age, breaks=breaks, right=FALSE, labels=tags)

In [ ]:
head(clean_unmatched_demo_df)
dim(clean_unmatched_demo_df)

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a fst file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- clean_unmatched_demo_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'unmatched_control_demo_df_11032023_AG.fst'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_fst(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.fst"), intern=T)

## Age Distribution

### Controls

In [ ]:
# Organize cohort by age distribution: 

age_counts <- select(clean_controls_demo_df, Count, Age_Group) %>% group_by(Age_Group)
age_counts <- count(age_counts, Age_Group)
colnames(age_counts) <- c('Age_Group','Count')
age_counts


x <- clean_controls_demo_df$Age
h<-hist(x, breaks=10, col="grey", xlab="Age",
   main="Histogram with Normal Curve")
xfit<-seq(min(x),max(x),length=40)
yfit<-dnorm(xfit,mean=mean(x),sd=sd(x))
yfit <- yfit*diff(h$mids[1:2])*length(x)
lines(xfit, yfit, col="blue", lwd=2)

### Cases

In [ ]:
# Organize cohort by age distribution:

age_counts <- select(clean_cases_demo_df, Count, Age_Group) %>% group_by(Age_Group)
age_counts <- count(age_counts, Age_Group)
colnames(age_counts) <- c('Age_Group','Count')
age_counts


x <- clean_cases_demo_df$Age
h<-hist(x, breaks=10, col="grey", xlab="Age",
   main="Histogram with Normal Curve")
xfit<-seq(min(x),max(x),length=40)
yfit<-dnorm(xfit,mean=mean(x),sd=sd(x))
yfit <- yfit*diff(h$mids[1:2])*length(x)
lines(xfit, yfit, col="blue", lwd=2)

### Unmatched

In [ ]:
# Organize cohort by age distribution:

age_counts <- select(clean_unmatched_demo_df, Count, Age_Group) %>% group_by(Age_Group)
age_counts <- count(age_counts, Age_Group)
colnames(age_counts) <- c('Age_Group','Count')
age_counts


x <- clean_unmatched_demo_df$Age
h<-hist(x, breaks=10, col="grey", xlab="Age",
   main="Histogram with Normal Curve")
xfit<-seq(min(x),max(x),length=40)
yfit<-dnorm(xfit,mean=mean(x),sd=sd(x))
yfit <- yfit*diff(h$mids[1:2])*length(x)
lines(xfit, yfit, col="blue", lwd=2)

## Gender Distribution

### Controls

In [ ]:
# Make a pie chart for sex assigned at birth

sex_counts <- select(clean_controls_demo_df, Count, Sex_at_Birth) %>% group_by(Sex_at_Birth)
sex_counts <- count(sex_counts, Sex_at_Birth)
colnames(sex_counts) <- c('Sex_at_Birth','Count')

sex_counts <- sex_counts %>% 
    mutate(Sex_at_Birth = ifelse(Sex_at_Birth == 'Not male, not female, prefer not to answer, or skipped',
                                'Not male, not female, prefer\nnot to answer, or skipped', Sex_at_Birth))

slices <- sex_counts$Count
lgd <- sex_counts$Sex_at_Birth
pct <- format(100*(sex_counts$Count)/(sum(sex_counts$Count)), digits = 2)
lbls <- paste(pct,"%",sep="") # ad % to labels

cols = RColorBrewer::brewer.pal(n = length(lgd), name = 'Set3')

sex_counts

par(mar=c(0,0,2,2))
pie(slices, lbls, main = 'sex assigned at birth', font=9, col=cols)
legend("topright", legend=lgd, fill=cols)

### Cases

In [ ]:
# Make a pie chart for sex assigned at birth

sex_counts <- select(clean_cases_demo_df, Count, Sex_at_Birth) %>% group_by(Sex_at_Birth)
sex_counts <- count(sex_counts, Sex_at_Birth)
colnames(sex_counts) <- c('Sex_at_Birth','Count')

sex_counts <- sex_counts %>% 
    mutate(Sex_at_Birth = ifelse(Sex_at_Birth == 'Not male, not female, prefer not to answer, or skipped',
                                'Not male, not female, prefer\nnot to answer, or skipped', Sex_at_Birth))

slices <- sex_counts$Count
lgd <- sex_counts$Sex_at_Birth
pct <- format(100*(sex_counts$Count)/(sum(sex_counts$Count)), digits = 2)
lbls <- paste(pct,"%",sep="") # ad % to labels

cols = RColorBrewer::brewer.pal(n = length(lgd), name = 'Set3')

sex_counts

par(mar=c(0,0,2,2))
pie(slices, lbls, main = 'sex assigned at birth', font=9, col=cols)
legend("topright", legend=lgd, fill=cols)

### Unmatched

In [ ]:
# Make a pie chart for sex assigned at birth

sex_counts <- select(clean_unmatched_demo_df, Count, Sex_at_Birth) %>% group_by(Sex_at_Birth)
sex_counts <- count(sex_counts, Sex_at_Birth)
colnames(sex_counts) <- c('Sex_at_Birth','Count')

sex_counts <- sex_counts %>% 
    mutate(Sex_at_Birth = ifelse(Sex_at_Birth == 'Not male, not female, prefer not to answer, or skipped',
                                'Not male, not female, prefer\nnot to answer, or skipped', Sex_at_Birth))

slices <- sex_counts$Count
lgd <- sex_counts$Sex_at_Birth
pct <- format(100*(sex_counts$Count)/(sum(sex_counts$Count)), digits = 2)
lbls <- paste(pct,"%",sep="") # ad % to labels

cols = RColorBrewer::brewer.pal(n = length(lgd), name = 'Set3')

sex_counts

par(mar=c(0,0,2,2))
pie(slices, lbls, main = 'sex assigned at birth', font=9, col=cols)
legend("topright", legend=lgd, fill=cols)

## Race Distribution

### Controls

In [ ]:
# Organize cohort by race and ancestry

race_counts <- select(clean_controls_demo_df, Count, Race) %>% group_by(Race)
race_counts <- count(race_counts, Race)
colnames(race_counts) <- c('Race','Count')
par(las = 1) # make label text perpendicular to axis
par(mar=c(3,15,3,1)) # increase y-axis margin


race_counts
barplot(race_counts$Count, main="Race and Ancestry", horiz = TRUE, 
        names.arg = race_counts$Race, cex.names = 0.8)

### Cases

In [ ]:
# Organize cohort by race and ancestry

race_counts <- select(clean_cases_demo_df, Count, Race) %>% group_by(Race)
race_counts <- count(race_counts, Race)
colnames(race_counts) <- c('Race','Count')
par(las = 1) # make label text perpendicular to axis
par(mar=c(3,15,3,1)) # increase y-axis margin


race_counts
barplot(race_counts$Count, main="Race and Ancestry", horiz = TRUE, 
        names.arg = race_counts$Race, cex.names = 0.8)

### Unmatched

In [ ]:
# Organize cohort by race and ancestry

race_counts <- select(clean_unmatched_demo_df, Count, Race) %>% group_by(Race)
race_counts <- count(race_counts, Race)
colnames(race_counts) <- c('Race','Count')
par(las = 1) # make label text perpendicular to axis
par(mar=c(3,15,3,1)) # increase y-axis margin


race_counts
barplot(race_counts$Count, main="Race and Ancestry", horiz = TRUE, 
        names.arg = race_counts$Race, cex.names = 0.8)

## Hispanic

### Controls

In [ ]:
# Organize cohort: Hispanic, Latino, or Spanish

hls_counts <- select(clean_controls_demo_df, Count, Hispanic) %>% group_by(Hispanic)
hls_counts <- count(hls_counts, Hispanic)
colnames(hls_counts) <- c('Hispanic','Count')

hls_graph <- hls_counts
hls_graph$Percentage <- format(100*(hls_graph$Count)/(sum(hls_graph$Count)),digits = 2)
slices <- hls_graph$Count
lgd <- hls_graph$Hispanic
pct <- hls_graph$Percentage
lbls <- paste(pct,"%",sep="") # ad % to labels

cols = RColorBrewer::brewer.pal(n = length(lgd), name = 'Set3')

hls_counts

par(mar=c(0,0,2,2))
pie(slices, lbls, main = 'Hispanic Latino or Spanish', font=9, col=cols)
legend("topright", legend=lgd, fill=cols)

### Cases

In [ ]:
# Organize cohort: Hispanic, Latino, or Spanish

hls_counts <- select(clean_cases_demo_df, Count, Hispanic) %>% group_by(Hispanic)
hls_counts <- count(hls_counts, Hispanic)
colnames(hls_counts) <- c('Hispanic','Count')

hls_graph <- hls_counts
hls_graph$Percentage <- format(100*(hls_graph$Count)/(sum(hls_graph$Count)),digits = 2)
slices <- hls_graph$Count
lgd <- hls_graph$Hispanic
pct <- hls_graph$Percentage
lbls <- paste(pct,"%",sep="") # ad % to labels

cols = RColorBrewer::brewer.pal(n = length(lgd), name = 'Set3')

hls_counts

par(mar=c(0,0,2,2))
pie(slices, lbls, main = 'Hispanic Latino or Spanish', font=9, col=cols)
legend("topright", legend=lgd, fill=cols)

### Unmatched

In [ ]:
# Organize cohort: Hispanic, Latino, or Spanish

hls_counts <- select(clean_unmatched_demo_df, Count, Hispanic) %>% group_by(Hispanic)
hls_counts <- count(hls_counts, Hispanic)
colnames(hls_counts) <- c('Hispanic','Count')

hls_graph <- hls_counts
hls_graph$Percentage <- format(100*(hls_graph$Count)/(sum(hls_graph$Count)),digits = 2)
slices <- hls_graph$Count
lgd <- hls_graph$Hispanic
pct <- hls_graph$Percentage
lbls <- paste(pct,"%",sep="") # ad % to labels

cols = RColorBrewer::brewer.pal(n = length(lgd), name = 'Set3')

hls_counts

par(mar=c(0,0,2,2))
pie(slices, lbls, main = 'Hispanic Latino or Spanish', font=9, col=cols)
legend("topright", legend=lgd, fill=cols)